In [1]:
include("util.jl");

## Day 1

In [2]:
depths = getinput(1) |> split .|> int

function count_ascending(arr)
    (d′ > d for (d, d′) in zip(arr, arr[2:end])) |> sum
end;

In [3]:
@time count_ascending(depths)

  0.095016 seconds (371.25 k allocations: 21.760 MiB, 15.82% gc time, 99.97% compilation time)


1532

In [4]:
sums3 = (depths[i:i+2] for i ∈ 1:(length(depths)-2)) .|> sum
@time count_ascending(sums3)

  0.000007 seconds (2 allocations: 15.766 KiB)


1571

## Day 2

In [5]:
dirs = Dict(
    "forward" => [1, 0],
    "down" => [0, 1],
    "up" => [0, -1]
)

function parse_row(row)
    dir, mag = split(row)
    return dirs[dir] * int(mag)
end

commands = split(getinput(2), "\n") .|> parse_row

pos = sum(commands)
pos, *(pos...)

([1832, 1172], 2147104)

In [6]:
pos = [0, 0]
aim = 0

for row in split(getinput(2), "\n")
    command, mag = split(row)
    if command == "forward"
       pos += [1, aim] * int(mag)
    else
        aim += (command == "down" ? 1 : -1) * int(mag)
    end    
end

pos, *(pos...)

([1832, 1116059], 2044620088)

## Day 3

In [7]:
rows = getinput(3) |> split .|> collect
mat = (hcat(rows...) .== '1')'
N = length(rows)
γ = mapslices(mat, dims=1) do col
    sum(col) > N // 2
end[:]
ϵ = γ .== 0

function base10(bitvec)
    str = convert.(Int, bitvec) |> join
    return parse(Int, str, base=2)
end

power = base10(γ) * base10(ϵ)

693486

In [10]:
function oxygen(mat, col)
    N = size(mat, 1)
    most_common = sum(mat[:,col]) / N >= 0.5
    idxs = filter(i -> mat[i, col] == most_common, 1:N)
    return mat[idxs,:]
end

m = mat
for i in 1:12
    m = oxygen(m, i)
end
base10(m[:])

933

In [15]:
function co2(mat, col)
    N = size(mat, 1)
    most_common = sum(mat[:,col]) / N >= 0.5
    idxs = filter(i -> mat[i, col] != most_common, 1:N)
    return mat[idxs,:]
end

m = mat
for i in 1:12
    if size(m, 1) == 1
        break;
    end
    m = co2(m, i)
end
base10(m[:])

3622

In [16]:
3622 * 933

3379326

## Day 4

In [26]:
numbers, boards... = split(getinput(4), "\n\n")

numbers = split(numbers, ",") .|> int

function parse_board(b)
    board = b |> split .|> int
    return reshape(board, 5, 5)
end
boards = boards .|> parse_board

board_hits = [zeros(Bool, 5, 5) for _ in 1:length(boards)]

completed_row(h) = any(mapslices(sum, h, dims=1).==5)
completed(h) = completed_row(h) || completed_row(h')


winner = nothing

for number in numbers
    for (board, hits) in zip(boards, board_hits)
        idx = findfirst(x -> x == number, board)
        if idx === nothing
            continue
        end
        hits[idx] = true
        if completed(hits)
            winner = board, hits, number
        end
    end

    if winner !== nothing
        break
    end
end

board, mask, number = winner
score = sum((mask .== 0) .* board) * number

69579

In [27]:
num = -1
lastwinner = -1
winners = []
board_hits = [zeros(Bool, 5, 5) for _ in 1:length(boards)]

for number in numbers
    for (b_idx, (board, hits)) in enumerate(zip(boards, board_hits))
        idx = findfirst(x -> x == number, board)
        if idx === nothing
            continue
        end
        hits[idx] = true
        if completed(hits) && b_idx ∉ winners
            push!(winners, b_idx)
            lastwinner = b_idx
        end
    end
    if length(winners) == length(boards)
        num = number
        break
    end
end

board, mask = boards[lastwinner], board_hits[lastwinner]
score = sum((mask .== 0) .* board) * num


14877

# Day 5

In [82]:
lines = getinput(5) |> spliton("\n")

coords = lines .|> spliton(" -> ")
to_coords(x) = int.(split(x, ',')) .+ 1
lines = reshape(vcat(coords...), 2, :) .|> to_coords

2×500 Matrix{Vector{Int64}}:
 [782, 722]  [335, 552]  [26, 154]   …  [43, 970]  [364, 689]  [646, 881]
 [782, 612]  [120, 552]  [766, 894]     [984, 29]  [364, 149]  [266, 501]

In [94]:
"""
    ordering(i, j)

1 if i < j
0 if i = j
-1 if i > j
"""
ordering(i, j) = i < j ? 1 : (i == j ? 0 : -1)

function displacement_unit_vector(a, b)
    x = ordering(a[1], b[1])
    y = ordering(a[2], b[2])
    return [x, y]
end

function solve(vec_allowed)
    grid = zeros(Int, 1000, 1000)
    for (pointa, pointb) in eachcol(lines)
        d = displacement_unit_vector(pointa, pointb)
        if !vec_allowed(d)
            continue
        end
        pointc = copy(pointa)
        while true
            grid[pointc...] += 1
            pointc != pointb || break
            pointc += d
        end
    end

    sum(grid .> 1)
end

solve(d -> sum(abs.(d)) == 1)

3990

In [95]:
solve(d -> true)

21305

# Day 6

In [224]:
starting = getinput(6) |> spliton(",") .|> int
starting .+= 1
ages = zeros(Int, 9)
for age in starting
    ages[age] += 1
end

function alive_after(starting_ages, days)
    ages = copy(starting_ages)
    for day in 1:days
        new = ages[1]
        ages = vcat(ages[2:end], [new])
        ages[7] += new
    end

    sum(ages)
end

alive_after(ages, 80)

395627

In [108]:
alive_after(ages, 256)

1767323539209

In [233]:
step_matrix = zeros(Int, 9, 9)
foreach(1:8) do i; step_matrix[i, i+1] = 1; end
step_matrix[[7,9],1] .= 1
# step_matrix[1,9] = 1
alive_after_imp(days) = (step_matrix^days * ages) |> sum
alive_after_imp(80), alive_after_imp(256)

(395627, 1767323539209)

# Day 7

In [177]:
inp = 7 |> getinput |> spliton(",") .|> int
N = maximum(inp)+1
occupancies = zeros(Int, N)
for i in inp
    occupancies[i+1] += 1
end

function cost_for_distance(d, fuel_for_distance)
    distances = (1:N) .- d .|> abs
    fuels = distances .|> fuel_for_distance
    return sum(fuels .* occupancies)
end

function solve(fuel_for_distance)
    costs = 1:N .|> d -> cost_for_distance(d, fuel_for_distance)
    return costs |> minimum
end

@time solve(n -> n)

  0.181652 seconds (400.48 k allocations: 105.023 MiB, 10.74% gc time, 80.31% compilation time)


345197

In [185]:
@time cost_for_distance(sort(inp)[500], n->n)

  0.078429 seconds (144.21 k allocations: 8.908 MiB, 11.39% gc time, 98.99% compilation time)


345199

In [178]:
triangle(n) = convert(Int, n * (n+1) / 2)

@time solve(triangle)

  0.151090 seconds (310.88 k allocations: 99.077 MiB, 18.36% gc time, 76.31% compilation time)


96361606

In [179]:
# faster - nice proof that the minimum lies within x̄ ± 0.5
@time begin m = sum(inp) / length(inp)
    candidates = m ± 0.5 .|> round .|> int
    costs = candidates .|> x -> cost_for_distance(x, triangle)
    minimum(costs)
end

  0.076791 seconds (163.10 k allocations: 10.010 MiB, 98.78% compilation time)


96361606

# Day 8

In [28]:
inp = 8 |> getinput |> spliton("\n") .|> replace(" | " => " ") .|> split
mat = sort.(hcat(inp...))
outputs = mat[11:14, :]

isunique(s) = length(s) ∈ [2, 3, 4, 7]

sum(isunique.(outputs))

318

In [123]:
difference(s1, s2) = [setdiff(s1, s2)...] |> String |> sort

#    t
# tl   tr
#    m
# bl   br
#    b

components = Dict(
    0 => [:top, :topleft, :topright, :bottomleft, :bottomright, :bottom],
    1 => [:topright, :bottomright],
    2 => [:top, :topright, :middle, :bottomleft, :bottom],
    3 => [:top, :topright, :middle, :bottomright, :bottom],
    4 => [:topleft, :middle, :topright, :bottomright],
    5 => [:top, :topleft, :middle, :bottomright, :bottom],
    6 => [:top, :topleft, :middle, :bottomright, :bottom, :bottomleft],
    7 => [:top, :topright, :bottomright],
    8 => [:top, :topleft, :topright, :middle, :bottomright, :bottomleft ,:bottom],
    9 => [:top, :topleft, :topright, :middle, :bottomright, :bottom],
)

function solve(example)
    digits, outputs = example[1:10], example[11:14]
    
    possibles = Dict((
        pos => "abcdefg" 
        for pos in (:top, :middle, :bottom, :topright, 
                    :topleft, :bottomright, :bottomleft)
    )...)

    function remove_singles()
        changed = false
        singles = filter(p -> length(p) == 1, collect(values(possibles)))
        for s in singles
            for (k, v) in possibles
                if length(v) > 1 && occursin(s, v)
                    possibles[k] = replace(v, s => "")
                    changed = true
                end
            end
        end
        
        !changed || remove_singles()
    end

    function only_possible(pos, vals)
        current = possibles[pos]
        possibles[pos] = filter(l -> l ∈ vals, current)
        remove_singles()
    end

    one, seven, four = map([2, 3, 4]) do l 
        i = findfirst(d -> length(d) == l, digits)
        digits[i]
    end 

    zero_six_nine = digits[findall(d -> length(d) == 6, digits)]

    (:topright, :bottomright) .|> p -> only_possible(p, one)
    only_possible(:top, difference(seven, one))
    (:topleft, :middle) .|> p -> only_possible(p, difference(four, one))
    (:topleft, :bottomright, :bottom) .|> p -> only_possible(p, intersect(zero_six_nine...) |> join)

    possibles 

    finals = map(0:9) do d
        comps = components[d]
        map(c -> possibles[c], comps) |> sort |> join
    end

    nums = map(1:4) do i
       findfirst(x -> outputs[i] == x, finals) - 1
    end

    tobase10(nums)
end

tobase10(arr) = sum(a * 10^i for (a, i) in zip(reverse(arr), 0:length(arr)-1))

solve.(eachcol(mat)) |> sum

996280

In [139]:
positions = vcat(values(components)...) |> unique
ids = Dict(p => "" for p in positions)
for (num, pos_s) in components
    foreach(pos_s) do pos
        ids[pos] *= string(num)
    end
end
# sort!.(values(ids))
ids

Dict{Symbol, String} with 7 entries:
  :topright    => "04279831"
  :middle      => "4562983"
  :bottom      => "0562983"
  :top         => "05627983"
  :bottomleft  => "0628"
  :topleft     => "045698"
  :bottomright => "045679831"

# Day 9

In [160]:
inp = 9 |> getinput |> split .|> spliton("")
mat = reshape(vcat(inp...), 100, :) .|> int
padded = ones(Int, 102, 102) * 9
padded[2:101, 2:101] = mat

function low_point(m)
    point, surroundings = m[5], m[[1,2,3,4,6,7,8,9]]
    return all(surroundings .> point)
end

mask = zeros(Bool, 100, 100)
for i ∈ 1:100, j ∈ 1:100
    p = padded[i:i+2, j:j+2]
    mask[i,j] = low_point(p)
end
sum(mask .* (mat.+1))

545

In [215]:
basins = zeros(Int, size(mat)...) - int.(mat .== 9)
sizes = Dict(0 => 0)

findnextbasin() = findfirst(basins .== 0)

compass = ([0, 1], [0, -1], [1, 0], [-1, 0]) .|> x -> CartesianIndex(x...)

function findneighbours(idx)
    res = []
    for dir in compass
        new_idx = idx + dir
        if 0 ∈ Tuple(new_idx) || 101 ∈ Tuple(new_idx)
            continue
        end
        if basins[new_idx] == 0
            push!(res, new_idx)
        end
    end
    return res
end

b_id = 1
while findnextbasin() !== nothing
    b_members = []
    b_frontier = Set()
    push!(b_frontier, findnextbasin())

    while length(b_frontier) > 0
        # @show b_frontier
        new_member = pop!(b_frontier)
        basins[new_member] = b_id
        push!(b_members, new_member)
        neighbours = [n for n in findneighbours(new_member) if n ∉ b_members]
        if length(neighbours) > 0
            push!(b_frontier, neighbours...)
        end
    end

    sizes[b_id] = length(b_members)
    b_id += 1 
end

all_sizes = sort(values(sizes) |> collect)
*(all_sizes[end-2:end]...)

950600

950600

In [186]:
b_frontier

1-element Vector{Any}:
 CartesianIndex(3, 1)

# Day 10

In [16]:
closing = ">< }{ ][ )(" |> split |> Dict

function process(incomplete, corrupted)
    return function(line)
        stack = []
        for char in line
            if char ∉ keys(closing)
                push!(stack, char)
            else
                other = pop!(stack)
                closing[char] == other || return corrupted(char)
            end
        end
        return incomplete(stack)
    end 
end

process (generic function with 1 method)

In [17]:
_scores = Dict(
    '>' => 25137,
    '}' => 1197,
    ']' => 57,
    ')' => 3
)
getscore = process(x -> 0, x -> _scores[x])

10 |> getinput |> spliton("\n") .|> getscore |> sum

166191

In [49]:
iscorrupted = process(x -> false, x -> true)

_scores = Dict(
    '>' => 4,
    '}' => 3,
    ']' => 2,
    ')' => 1
)

opening = Dict(value => key for (key, value) in closing)

function completion_score(str)
    score = 0
    for char in str
        score *= 5
        score += _scores[char]
    end
    score
end

function completion_string(stack)
    map(x -> opening[x], reverse(stack))
end


all_lines = 10 |> getinput |> spliton("\n") 

getscore = process(
    x -> x |> completion_string |> completion_score,
    x -> 0
)

all_scores = all_lines .|> getscore 
filter!(x -> x != 0, all_scores)
sort(all_scores)[round(Int, end/2)]

1152088313